# Notebook Title

## Setup Python and R environment
you can ignore this section

In [62]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [64]:
%%R

# My commonly used R imports

require('tidyverse')

## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [95]:
import pandas as pd

df = pd.read_csv("mental_health_facilities_nyc_with_census.csv")  
df.head()

,name1,name2,street1,street2,city,state,zip,phone,intake1,intake2,...,service_code_info,full_address,location,latitude,longitude,GEOID,STATE,COUNTY,TRACT,BLOCK
0,Medstar Harbor Hospital,Behavioral Health,3001 South Hanover Street,Suite 164,Brooklyn,MD,21225,410-350-7550,NaN,NaN,...,MH * OP PHDT * PSY * ARIPI CLOZA OLANZ OLANZF ...,"3001 South Hanover Street, Brooklyn, NY 21225",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Astor Servs for Children and Families,Astor Day Treatment Program,516 East Tremont Avenue,NaN,Bronx,NY,10457,347-978-2450,929-285-3917 x1096,NaN,...,MH SUMH * PHDT * PH * CHLOR HALOP PERPH ANTPYC...,"516 East Tremont Avenue, Bronx, NY 10457","516, East Tremont Avenue, East Tremont, The Br...",40.846700,-73.896822,3.600504e+14,36.0,5.0,39500.0,4001.0
2,Astor Servs for Children and Families,Highbridge Clinic,1419 Shakespeare Avenue,1st Floor,Bronx,NY,10452,718-231-3400,718-732-7080 x0,NaN,...,SA MH SUMH * OP * OMH * NSC ANTPYCH * CBT CFT ...,"1419 Shakespeare Avenue, Bronx, NY 10452","1419, Shakespeare Avenue, High Bridge, The Bro...",40.842533,-73.921238,3.600502e+14,36.0,5.0,21302.0,3000.0
3,Astor Servs for Children and Families,Lawrence F Hickey Center,4010 Dyre Avenue,NaN,Bronx,NY,10466,845-515-3000,718-515-3000,NaN,...,MH * PHDT * PH * ANTPYCH * CBT CFT GT IPT TELE...,"4010 Dyre Avenue, Bronx, NY 10466","Public School 15, 4010, Dyre Avenue, Parkside,...",40.890946,-73.830730,3.600505e+14,36.0,5.0,45600.0,2002.0
4,Astor Servs for Children and Families,Tilden Clinic,750 Tilden Street,NaN,Bronx,NY,10467,718-231-3400,NaN,NaN,...,SA MH SUMH * OP * OMH * ANTPYCH * CBT CFT DBT ...,"750 Tilden Street, Bronx, NY 10467","750, Tilden Street, Williams Bridge, The Bronx...",40.876680,-73.862771,3.600504e+14,36.0,5.0,38000.0,4006.0


## 👉 Grab Census Data

1. loading the Census API key

In [96]:
import dotenv

dotenv.load_dotenv('api.env')

api_key = dotenv.get_key('api.env', 'CENSUS_API_KEY')

print(api_key is not None)  


True


In [97]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [98]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
11 B07011

In [121]:
%%R
library(tidycensus)
library(dplyr)

nyc_census_data <- get_acs(
  geography = "zcta",  
  variables = c(
    population = "B01003_001",
    med_inc = "B19013_001",
    white = "B02001_002",
    black = "B02001_003",
    native = "B02001_004",
    asian = "B02001_005",
    total_race = "B02001_001",
    poverty_total = "B17001_001",
    poverty_below = "B17001_002"
  ),
  year = 2022,
  survey = "acs5",
  output = "wide"
)


Getting data from the 2018-2022 5-year ACS


In [122]:
%%R
nyc_census_data <- nyc_census_data %>%
  filter(GEOID %in% nyc_mental_health$zip)


In [123]:
%%R
nyc_census_data <- nyc_census_data %>%
  mutate(
    white_pct = 100 * whiteE / total_raceE,
    black_pct = 100 * blackE / total_raceE,
    native_pct = 100 * nativeE / total_raceE,
    asian_pct = 100 * asianE / total_raceE,
    poverty_rate_pct = 100 * poverty_belowE / poverty_totalE
  )


In [124]:
%%R
colnames(nyc_census_data)

 [1] "GEOID"            "NAME"             "populationE"      "populationM"     
 [5] "med_incE"         "med_incM"         "whiteE"           "whiteM"          
 [9] "blackE"           "blackM"           "nativeE"          "nativeM"         
[13] "asianE"           "asianM"           "total_raceE"      "total_raceM"     
[17] "poverty_totalE"   "poverty_totalM"   "poverty_belowE"   "poverty_belowM"  
[21] "white_pct"        "black_pct"        "native_pct"       "asian_pct"       
[25] "poverty_rate_pct"


In [125]:
%%R


require('tidycensus')
require('tidyverse')

## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [126]:
%%R
nyc_census_data

# A tibble: 64 × 25
   GEOID NAME     populationE populationM med_incE med_incM whiteE whiteM blackE
   <chr> <chr>          <dbl>       <dbl>    <dbl>    <dbl>  <dbl>  <dbl>  <dbl>
 1 10001 ZCTA5 1…       27004        1827   106509     9423  15428   1696   2355
 2 10002 ZCTA5 1…       76518        2894    43362     5737  23951   1688   6785
 3 10003 ZCTA5 1…       53877        2579   152863    10149  36515   2428   2899
 4 10004 ZCTA5 1…        4579         926   232543    14137   2653    605    266
 5 10009 ZCTA5 1…       58418        3009    83344     8153  32738   2376   4477
 6 10010 ZCTA5 1…       32410        2681   150288    23588  21682   2315   2753
 7 10011 ZCTA5 1…       50772        2368   145934    17135  37319   2119   2345
 8 10014 ZCTA5 1…       29461        1737   147267    15234  24159   1514    650
 9 10016 ZCTA5 1…       54369        3189   145864    12565  34711   2443   2603
10 10017 ZCTA5 1…       14486        1498   139964    25136   9949   1321    461
# ℹ 54 m

In [127]:
%%R 
library(readr)
library(dplyr)
library(stringr)
mental_health_facilities_with_census <- read_csv("mental_health_facilities_nyc_with_census.csv")

nyc_mental_health <- mental_health_facilities_with_census %>%
  filter(str_detect(city, "New York|Bronx|Brooklyn|Staten Island|Queens|Manhattan") & state == "NY")

nyc_mental_health



Rows: 125 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (14): name1, name2, street1, street2, city, state, phone, intake1, intak...
dbl  (6): zip, latitude, longitude, GEOID, STATE, BLOCK
lgl  (2): intake1a, intake2a

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 124 × 22
   name1  name2 street1 street2 city  state   zip phone intake1 intake2 intake1a
   <chr>  <chr> <chr>   <chr>   <chr> <chr> <dbl> <chr> <chr>   <chr>   <lgl>   
 1 Astor… Asto… 516 Ea… <NA>    Bronx NY    10457 347-… 929-28… <NA>    NA      
 2 Astor… High… 1419 S… 1st Fl… Bronx NY    10452 718-… 718-73… <NA>    NA      
 3 Astor… Lawr… 4010 D… <NA>    Bronx NY    10466 845-… 718-51… <NA>    NA      
 4 Astor… Tild… 750 Ti… <NA>    Bronx NY    10467 718-… <NA>    <NA>    NA      
 5 BASIC… <NA>  915 We… 1st Fl… Bronx NY    10459 646-…

In [130]:
%%R
library(dplyr)

nyc_census_data <- nyc_census_data %>%
  mutate(ZIP = as.character(GEOID))  

nyc_mental_health <- nyc_mental_health %>%
  mutate(zip = as.character(zip))

merged_df <- inner_join(nyc_census_data, nyc_mental_health, by = c("ZIP" = "zip"))

# Check result
dim(merged_df)
head(merged_df)


# A tibble: 6 × 48
  GEOID.x NAME    populationE populationM med_incE med_incM whiteE whiteM blackE
  <chr>   <chr>         <dbl>       <dbl>    <dbl>    <dbl>  <dbl>  <dbl>  <dbl>
1 10001   ZCTA5 …       27004        1827   106509     9423  15428   1696   2355
2 10001   ZCTA5 …       27004        1827   106509     9423  15428   1696   2355
3 10001   ZCTA5 …       27004        1827   106509     9423  15428   1696   2355
4 10002   ZCTA5 …       76518        2894    43362     5737  23951   1688   6785
5 10002   ZCTA5 …       76518        2894    43362     5737  23951   1688   6785
6 10002   ZCTA5 …       76518        2894    43362     5737  23951   1688   6785
# ℹ 39 more variables: blackM <dbl>, nativeE <dbl>, nativeM <dbl>,
#   asianE <dbl>, asianM <dbl>, total_raceE <dbl>, total_raceM <dbl>,
#   poverty_totalE <dbl>, poverty_totalM <dbl>, poverty_belowE <dbl>,
#   poverty_belowM <dbl>, white_pct <dbl>, black_pct <dbl>, native_pct <dbl>,
#   asian_pct <dbl>, poverty_rate_pct <dbl>, ZIP

In [131]:
%%R
write.csv(merged_df, "merged.csv", row.names = FALSE)